## Second deduplication

This notebook begins with **manifestationmeta.tsv,** and moves toward a smaller dataset that *aspires* to contain only one copy of each "work," in [FRBR terminology.](https://en.wikipedia.org/wiki/Functional_Requirements_for_Bibliographic_Records) 

However, the key word there is "aspires." We actually rely on a probabilistic model that is known to be wrong about 12% of the time. The model predicts the probability that two records are "the same work," using evidence that includes the similarity of their authors and titles in metadata, but also the degree of similarity between *their texts,* as measured through cosine similarity on extracted features.

I've set the probability threshold at 66% to be cautious about collapsing works together. So when the model makes an error it will usually (7%) go wrong by saying that two works are different, and more rarely (5%) mistakenly claim they are the same.

**Note** this process is not completely reproducible from this notebook alone; it involves 4GB of data about extracted features that I have not uploaded to the GitHub repo. However if you consult **../get_EF** you can see how to download and process that data yourself. Fair warning: it took ~30hrs of processing.

In [21]:
import pandas as pd
from difflib import SequenceMatcher
from collections import Counter
import unicodedata
import math, random, pickle
import statsmodels.api as sm
from scipy import spatial
from sklearn.preprocessing import StandardScaler

### create blocks

We start by grouping volumes into "blocks." This is purely a time-reduction step, to avoid useless comparisons of very different volumes. Each block is identified by the first six characters of the author's name.

This strategy does unfortunately mean that the first few characters of names become very important, which is why I made some effort to standardize naming in the first deduplication notebook -- moving e.g. "sir" and "mrs" to the end of the name. More could probably be done here: names like "Du Maurier" and "Van Dyck" are potentially tricky.

We group volumes in "blocks" identified by the first six letters of the author's name. But we also group these blocks into 26 larger groups identified by their first initial. The reason for this is that we may need to parallelize processing and divide data into chunks.

In [16]:
meta = pd.read_csv('../manifestationmeta.tsv', sep = '\t', low_memory = False, index_col = 'docid')

blocks = dict()

for idx in meta.index:
    name = meta.loc[idx, 'author']
    if pd.isnull(name) or len(name) < 3:
        name = 'nan'
    else:
        name = unicodedata.normalize('NFC', name.lower())
    
    if len(name) < 6:
        blockcode = name
    else:
        blockcode = name[0:6]
    
    initial = blockcode[0]
    if not initial.isalpha() or ord(initial) > 128:
        initial = 'x'
    
    if not initial in blocks:
        blocks[initial] = dict()
    
    if not blockcode in blocks[initial]:
        blocks[initial][blockcode] = set()
    
    blocks[initial][blockcode].add(idx)

In [17]:
print('blocks: ', len(blocks))
allcodes = 0
for b, block in blocks.items():
    allcodes += len(block)
print('all codes: ', allcodes)
print('all volumes: ', len(meta.index))

blocks:  26
all codes:  22346
all volumes:  176623


### Dividing up the text data

I downloaded the HathiTrust extracted features for these 176,000 volumes and parsed them using **../get_EF/parsefeaturejsons.py** to produce a matrix where each row is a volume, and the top 1000 features are columns. This ends up being 4GB of data, which is a bit whopping to manipulate in pandas. Plus, I may need to parallelize the processing on different machines. So I'm going to divvy up the matrix.

While I'm doing that, I'm also going to sneakily do a couple of other things. First, I'm going to center and scale each of these matrices. (I.e., subtract column mean from each column, and divide by stddev.) The matrices won't all have exactly the same scale, but I don't think that's mission-critical.

Second, I'm going to add "group" rows to the matrices in cases where a volume belongs to a multi-volume record. This is a tricky aspect of textual similarity. Say I'm comparing a one-volume Middlemarch from 1960 to a 3-volume edition in 1881. "Oh," my program says, "this 1960 volume doesn't match the first volume of 1881." Well, no, of course it doesn't, because that's just the first volume, duh! To avoid that problem, we need to create a second row that sums all the evidence for the volumes. Here we do that by taking the mean of the volumes — since we're looking at frequencies rather than absolute counts, that's roughly adequate.

In [28]:
matrix1 = pd.read_csv('/Volumes/TARDIS/work/ef/ficmatrix/featurematrix1.csv', index_col = 'docid')
matrix2 = pd.read_csv('../data/featurematrix.csv', index_col = 'docid')
inmat1 = set(matrix1.index)
inmat2 = set(matrix2.index)

def probablymatch(str1, str2):
    
    m = SequenceMatcher(None, str1, str2)
    match = m.real_quick_ratio()
    if match > 0.75:
        match = m.ratio()
    
    return match

def find_groups(df):
    global meta
    groupvecs = dict()
    
    for d in df.index:
        record = int(meta.loc[d, 'recordid'])
        title = str(meta.loc[d, 'shorttitle'])
        thisrec = meta.loc[meta.recordid == record, : ]
        matching = []
        
        for idx in thisrec.index:
            thistitle = str(thisrec.loc[idx, 'shorttitle'])
            if thistitle == title or probablymatch(thistitle, title) > 0.9:
                matching.append(idx)

        if len(matching) > 1 and len(matching) < 6:
            matchvec = df.loc[matching, : ].mean(axis = 0)
            newidx = d + "group"
            groupvecs[newidx] = matchvec
    
    return pd.DataFrame.from_dict(groupvecs, orient = 'index')

for initial, block in blocks.items():
    allvols = set()
    for code, vols in block.items():
        allvols = allvols.union(vols)
    group1 = allvols.intersection(inmat1)
    df1 = matrix1.loc[group1, : ]
    group2 = allvols.intersection(inmat2)
    df2 = matrix2.loc[group2, : ]
    df = pd.concat([df1, df2])
    print(initial, df.shape)
    
    # let's scale the matrix
    scaler = StandardScaler()
    scaler.fit(df)
    scaled = scaler.transform(df)
    df = pd.DataFrame(scaled, index = df.index)
    
    # augment the matrix with group rows
    groupeddf = find_groups(df)
    df = pd.concat([df, groupeddf])
    print(df.shape)
    print()
    outfile = '/Volumes/TARDIS/work/ef/ficmatrix/matrix_' + initial + '.csv'
    df.to_csv(outfile)
        
        

s (15892, 1000)
(17604, 1000)

p (6971, 1000)
(7686, 1000)

d (10052, 1000)
(11092, 1000)

q (370, 1000)
(385, 1000)

o (3011, 1000)
(3398, 1000)

x (611, 1000)
(668, 1000)

y (919, 1000)
(1040, 1000)

i (1017, 1000)
(1117, 1000)

e (3700, 1000)
(4232, 1000)

w (8923, 1000)
(9658, 1000)

v (1881, 1000)
(1968, 1000)

k (5529, 1000)
(5924, 1000)

n (11023, 1000)
(12358, 1000)

r (6713, 1000)
(7529, 1000)

t (7156, 1000)
(8209, 1000)

l (8268, 1000)
(9291, 1000)

m (14806, 1000)
(16348, 1000)

c (12850, 1000)
(14213, 1000)

z (663, 1000)
(689, 1000)

h (12699, 1000)
(13735, 1000)

a (6517, 1000)
(6948, 1000)

u (449, 1000)
(456, 1000)

f (6309, 1000)
(6949, 1000)

b (16544, 1000)
(18054, 1000)

g (8632, 1000)
(9529, 1000)

j (3996, 1000)
(4420, 1000)



### Training the model

We need to train a model of similarity between volumes, and save the model.

Let's start by reading in the relevant data.

In [20]:
data = pd.read_csv('fulltrainingdata.tsv', sep = '\t')
X = data[['titlematch', 'cossim', 'hasworks']]
y = data['groundtruth']

# Now actually train the model

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.321825
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:            groundtruth   No. Observations:                 1109
Model:                          Logit   Df Residuals:                     1106
Method:                           MLE   Df Model:                            2
Date:                Sat, 05 May 2018   Pseudo R-squ.:                  0.4984
Time:                        17:00:28   Log-Likelihood:                -356.90
converged:                       True   LL-Null:                       -711.53
                                        LLR p-value:                9.724e-155
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
titlematch     3.1700      0.177     17.937      0.000       2.824       3.516
cossim        -7.8491      0.

#### Feature interpretation

**titlematch** is the fuzzy similarity between the titles of two volumes
**cossim** is the cosine similarity (or really divergence) between their texts
**hasworks** is a binary variable, either 0 or 1. It's 1 for comparisons where either title contains the word "works." This turned out to be very important, because we usually don't want to consider volumes of "Collected Works" as a match (if they lack shorter titles), and there are a lot of such volumes.

#### save to file

Now let's save the result to file, so that we can load it later if needed. Statsmodels have a convenient "save" function, so that we don't need to use the pickle module directly.

In [23]:
result.save('logistic_model_of_similarity.pickle')

In [40]:
testdf = pd.DataFrame({'titlematch': 0.95, 'cossim': .1, 'hasworks': 0}, index = ['test'], dtype = 'float')
testdf = testdf[['titlematch', 'cossim', 'hasworks']]
print(result.predict(testdf))

test    0.902612
dtype: float64


In [24]:
del matrix1
del matrix2

### Load a block and check for matches

Generally, the strategy here is to loop through each block, comparing each record to all the other records in the block. For each comparison, we first check a few basic thresholds (author similarity and title similarity must be > 0.8). If the connection passes those thresholds, we pass title similarity, cosine similarity of texts, and "hasworks" to the model.

For each volume, we keep a record of all the other volumes that match it. We can later transform this dictionary of *edges* into a list of *connected components*.


In [86]:
def probablymatch(str1, str2):
    '''Runs a quick check, and a better check if the upper bound on
    quick check suggests a better check is needed.'''
    m = SequenceMatcher(None, str1, str2)
    match = m.real_quick_ratio()
    if match > 0.75:
        match = m.ratio()
    
    return match

def cleanstring(astring, cap):
    astring = astring.replace(';', '')
    astring = astring.replace(':', '')
    astring = astring.lower()
    if len(astring) > cap:
        astring = astring[0 : cap]
    return astring

def has_works(title1, title2):
    ''' Returns 1 if either title in a pair has the word "works".'''
    words1 = title1.lower().split()
    words1 = [x.strip(',. ') for x in words1]
    words2 = title2.lower().split()
    words2 = [x.strip(',. ') for x in words2]
    
    if 'works' in words1 or 'works' in words2:
        return 1
    else:
        return 0

def calculate_cossim(doc1, doc2, df, inmatrix):
    ''' Calculates cosine similarity between two volumes, and between the
    larger groups of vols they belong to, if those groups exist.
    '''
    if doc1 in inmatrix and doc2 in inmatrix:
        vec1 = df.loc[doc1, : ]
        vec2 = df.loc[doc2, : ]
        cossimA = spatial.distance.cosine(vec1, vec2)
        
        doc1groupidx = doc1 + 'group'
        doc2groupidx = doc2 + 'group'
        
        if doc1groupidx in inmatrix:
            grouped1 = df.loc[doc1groupidx, : ]
            cossimB = spatial.distance.cosine(grouped1, vec2)
        else:
            cossimB = 100
        
        if doc2groupidx in inmatrix:
            grouped2 = df.loc[doc2groupidx, : ]
            cossimC = spatial.distance.cosine(vec1, grouped2)
        else:
            cossimC = 100
        
        if cossimB < 100 and cossimC < 100:
            cossimD = spatial.distance.cosine(grouped1, grouped2)
        else:
            cossimD = 100
        
        cossim = min(cossimA, cossimB, cossimC, cossimD)
        
    else:
        cossim = 0.2151
        # This was the mean in our training set, and will be used in
        # place of NA for comparisons where either vol is missing.
    
    return cossim
    
def get_matches(initial, blocks, model):
    
    block = blocks[initial]
    
    # get the text data for this block
    dataname = '/Volumes/TARDIS/work/ef/ficmatrix/matrix_' + initial + '.csv'
    textmatrix = pd.read_csv(dataname, index_col = 'docid')
    inmatrix = set(textmatrix.index)
    
    matches = dict()
    repeats = 0
    
    for code, volset in block.items():
        
        vols = list(volset)
        
        already_checked = dict()
        titledict = dict()
        authdict = dict()
    
        # we clean all the titles and authors in the vols before 
        # attempting to match; otherwise you end up doing
        # n x n cleaning operations.
        
        # we also initialize matches
    
        for b in vols:
            if b not in matches:
                matches[b] = set()
            else:
                repeats += 1
                # that shouldn't happen
                
            auth = meta.loc[b, 'author']
            if pd.isnull(auth) or len(auth) < 3:
                auth = 'cannot-match'
            else:
                auth = cleanstring(auth, 25)

            title = meta.loc[b, 'shorttitle']
            if pd.isnull(title) or len(title) < 3:
                title = 'cannot-match'
            else:
                title = cleanstring(title, 35)

            titledict[b] = title
            authdict[b] = auth

        for idx, b1 in enumerate(vols):
            
            for b2 in vols[idx + 1 : ]:
                
                auth1 = authdict[b1]
                auth2 = authdict[b2]
                title1 = titledict[b1]
                title2 = titledict[b2]

                if auth1 == 'cannot-match' or auth2 == 'cannot-match':
                    continue
                    
                if title1 == 'cannot-match' or title2 == 'cannot-match':
                    continue

                if auth1 == auth2:
                    authormatch = 1.0
                else:
                    authormatch = probablymatch(auth1, auth2)
                    if authormatch < 0.8:
                        # we insist on more similarity in authors
                        continue

                if title1 == title2:
                    titlematch = 1.0
                else:
                    titlematch = probablymatch(title1, title2)
                    if titlematch < 0.8:
                        # we insist on more similarity in titles
                        continue

                cossim = calculate_cossim(b1, b2, textmatrix, inmatrix)
                hasworks = has_works(title1, title2)

                testdf = pd.DataFrame({'titlematch': titlematch, 'cossim': cossim, 'hasworks': hasworks}, index = ['test'], dtype = 'float64')
                testdf = testdf[['titlematch', 'cossim', 'hasworks']]
                probability = float(model.predict(testdf))
                
                if probability < 0.66:
                    continue
                else:
                    matches[b1].add(b2)
                    matches[b2].add(b1)
                    
    if repeats > 0:
        print('repeats ', repeats)
    return matches              

### Connect components

The previous function gave us a dictionary where each volume is linked to a set of volumes that match it. This is in essence a data structure of *edges* in a graph.

Now we need to transform that structure into a list of *connected components*. Basically, like so:

![caption](files/connected.png)

Image credit: [Sebastian Thomas.](https://www.mathworks.com/matlabcentral/fileexchange/46457-splitting-a-network-into-connected-components)

In [91]:
def dfs(vertex, matchdict, visited, components, component_ctr):
    ''' Depth-first search algorithm. '''
    visited.add(vertex)
    components[component_ctr].add(vertex)
    for link in matchdict[vertex]:
        if link not in visited:
            dfs(link, matchdict, visited, components, component_ctr)
            
def connect_components(matchdict):
    ''' Visit each vertex. If not yet visited, create a new component, and do 
    depth-first search on the vertex, adding all linked vertices to the new
    component.
    '''
    
    visited = set()
    components = []
    component_ctr = 0
    
    for vertex, links in matchdict.items():
        if vertex not in visited:
            components.append(set())
            dfs(vertex, matchdict, visited, components, component_ctr)
            component_ctr += 1
    
    return components
    
matches = get_matches('q', blocks, result)
components = connect_components(matches)
print(len(components))
    

285


In [52]:
initialist = []
for initial, block in blocks.items():
    initialist.append((len(block), initial))

initialist.sort()
initialist = [x[1] for x in initialist]
initialist

['q',
 'u',
 'y',
 'z',
 'i',
 'x',
 'j',
 'v',
 'o',
 'e',
 'n',
 'f',
 'w',
 't',
 'r',
 'p',
 'a',
 'k',
 'g',
 'l',
 'd',
 'h',
 'c',
 'b',
 'm',
 's']

In [92]:
components = []
for initial in initialist:
    print(initial, len(blocks[initial]))
    matches = get_matches(initial, blocks, result)
    components.extend(connect_components(matches))
    print(len(components))

q 62
285
u 135
680
y 148
1365
z 175
1889
i 245
2673
x 353
3198
j 358
5938
v 460
7504
o 538
9845
e 547
12438
n 548
23022
f 730
27781
w 777
34446
t 853
39084
r 953
44085
p 1025
49506
a 1038
54336
k 1096
58583
g 1115
65073
l 1177
70857
d 1233
77571
h 1297
86565
c 1450
95713
b 1988
108002
m 1992
119216
s 2053
130463


### A little exploratory description

E.g., how many groups do we have? How big is the biggest?

In [96]:
print('We have ' + str(len(components)) + " different components.")

maxsize = 0
for c in components:
    if len(c) > maxsize:
        maxsize = len(c)
        for ex_biggest in c:
            break
print("Of which the biggest contains " + str(maxsize) + " vols.")
print(meta.loc[ex_biggest, : ])

We have 130463 different components.
Of which the biggest contains 483 vols.
oldauthor                                           Scott, Walter
author                                         Scott, Walter, Sir
authordate                                             1771-1832.
inferreddate                                                 1829
latestcomp                                                   1829
datetype                                                        m
startdate                                                    1829
enddate                                                      1833
imprint                              Edinburgh;R. Cadell;1829-33.
imprintdate                                             1829-1833
contents        XLVIII. [Castle Dangerous] The surgeon's daugh...
genres                                                    Fiction
subjects                                                      NaN
geographics                                                   NaN

### Now the actual deduplication

In principle, generally, we want to take one volume from each group of volumes that have matching titles and authors. And in general we want to take the earliest volume, so our resulting dataset will be dated as close as possible to dates of first publication.

However, there are complicating cases. What if, for instance, the earliest instance of a novel is a Victorian three-decker edition? That's going to happen pretty often. In that case, we don't want to take *just one volume* from the group; we want all three volumes of the earliest edition. So we need a new rule: take all volumes sharing the *recordid* of the earliest volume. That will get all three volumes of a three-volume edition.

But we confront yet another complication! Volumes grouped by a recordid are sometimes three volumes of a single work. But often they are, say, 28 volumes in the *Collected Works of Scott.* All sharing a single record id, but not all the same fictional work. Maybe some of the longer novels are spread across 2 or three volumes, but many of the volumes represent a single novel. This gets bloody complicated.

So our *new* rule is: find the earliest volume. Get its record id. Find all volumes sharing that record id (all volumes in the same set). Then take all the volumes that share the same *short title*. If we have been able to identify vols 11 and 12 as *Ivanhoe,* this will get just 11 and 12. However, if we haven't been able to identify titles beyond *Collected Works of Scott,* we'll get all 28 vols! So the final rule is, ignore cases where we recover more than five vols sharing the same recordid. We suspect these are collected works.

As we do this, we are going to want to keep track of the number of copies of a volume that have been collapsed into a single deduplicated record. We'll use a column of "instances" created in the earlier stage of deduplication; this counts vols that had the same recordid+volnum. We'll further aggregate that into "copies": vols that had the same author/title. Moreover, since we may want to distinguish *contemporary* popularity from later canonicity, we're going to keep track of this in two different ways: a general column of copies and a column of copies-published-within-25-yrs of our first example.

In [98]:
selected = []
ignored = []
errors = 0
authtitlecopies = dict()
copiesin25yrs = dict()
authorsets = []

ctr = 0
for g in components:
    ctr += 1
    if ctr % 10000 == 1:
        print(ctr)
    
    # Some groups contain only a single volume.
    if len(g) == 1:
        for e in g:
            break
        selected.append(e)
        authtitlecopies[e] = int(meta.loc[e, 'instances'])
        copiesin25yrs[e] = authtitlecopies[e]
        # For a single volume, all these quantities will be the same.
        continue
        
    if len(g) < 1:
        errors += 1
        continue
    
    earliest = ''
    earliestdate = 2100
    instancectr = Counter()
    authorset = set()
    
    for element in g:
        date = meta.loc[element, 'inferreddate']
        copies = int(meta.loc[element, 'instances'])
        auth = meta.loc[element, 'author']
        if not pd.isnull(auth):
            authorset.add(auth)
        
        if pd.isnull(date) or int(date) == 0:
            date = 2100
        else:
            date = int(date)
        
        instancectr[date] += copies
        
        if earliestdate == 2100 or date < earliestdate:
            earliestdate = date
            earliest = element
            if earliestdate < 1700:
                earliestdate = 2100
                # don't reward dubious dates
                
    # different authnames?
    if len(authorset) > 1:
        authorsets.append(authorset)
        
    # now let's add up those copies
    allcopies = 0
    copiesin25yrsofearliest = 0
    
    for date, count in instancectr.items():
        allcopies += count
        if date < (earliestdate + 25):
            copiesin25yrsofearliest += count
            
    record = meta.loc[earliest, 'recordid']
    title2match = str(meta.loc[earliest, 'shorttitle'])

    matching = []

    thisrec = meta.loc[meta.recordid == record, : ]
    for idx in thisrec.index:
        thistitle = str(thisrec.loc[idx, 'shorttitle'])
        match = probablymatch(title2match, thistitle)
        if match > 0.9:
            matching.append(idx)
    
    if len(matching) < 6:
        selected.extend(matching)
        for m in matching:
            authtitlecopies[m] = allcopies
            copiesin25yrs[m] = copiesin25yrsofearliest
    else:
        ignored.append((title2match, record))
        
print(errors)          

1
10001
20001
30001
40001
50001
60001
70001
80001
90001
100001
110001
120001
130001
0


### Some exploratory description

For instance, how many records did we select? How many groups of vols were ignored?

Note also that I quietly prune duplicate docids from the **selected** list. The algorithm above permits some duplication to happen, though it's not huge.


In [99]:
print(len(selected))

# get rid of duplicates
selected = list(set(selected))
print(len(selected))


137017
136991


In [100]:
print(len(ignored))
print(len(authorsets))

382
396


In [102]:
# Let's write the ignored records to file

with open('ignoredgroups.tsv', mode = 'w', encoding = 'utf-8') as f:
    for title, record in ignored:
        f.write(title + '\t' + str(record) + '\n')

In [103]:
# Also the groups

with open('allgroups.tsv', mode = 'w', encoding = 'utf-8') as f:
    for g in components:
        f.write('\t'.join(g) + '\n')

In [104]:
# And the authorsets

authorsets = set([tuple(x) for x in authorsets])
print(len(authorsets))
# reduce duplication

with open('authorsets.tsv', mode = 'w', encoding = 'utf-8') as f:
    for s in authorsets:
        f.write('\t'.join(s) + '\n')

297


### Now actually produce and write the dataframe

All of our effort so far has gone into selecting a list of indices that will be retained. Now we have to use those indices to actually produce a new dataframe.

In [105]:
# like so

deduped = meta.loc[selected, : ]

In [106]:
deduped.head()

,oldauthor,author,authordate,inferreddate,latestcomp,datetype,startdate,enddate,imprint,imprintdate,...,locnum,oclc,place,recordid,enumcron,volnum,title,parttitle,shorttitle,instances
docid,,,,,,,,,,,,,,,,,,,,,
mdp.39015048438470,"White, Hervey","White, Hervey",1866-1944.,1913,1913,s,1913,,"Woodstock, N. Y.;The Maverick press;1913.",1913,...,PZ3.W5837Ho,819785,nyu,325093,NaN,NaN,"The house in the road; | a fantasy of truth, |...",NaN,The house in the road; a fantasy of truth,1
pst.000033219158,"Tierney, James","Tierney, James",NaN,1997,1997,s,1997,,"Merrylands, NSW, Australia|Newman Centre Publi...",1997,...,NaN,38758319,xx,7509962,T579bushbu 1997,NaN,Bush boys and bush rangers / | $c: by James Ti...,NaN,Bush boys and bush rangers,1
uc1.b4368903,"Tuttle, Anthony","Tuttle, Anthony",NaN,1972,1972,s,1972,,"Garden City, N.Y.|Doubleday|1972.",1972,...,NaN,240543,nyu,9510251,NaN,NaN,Songs from the night before.,NaN,Songs from the night before,1
uc1.b4357916,"Asch, Sholem","Asch, Sholem",1880-1957.,1946,1946,s,1946,,New York|G. P. Putnam's sons|1946,1946,...,NaN,325381,nyu,983714,NaN,NaN,"East river, | a novel. | $c: Translation by A....",NaN,"East river, a novel",3
wu.89044674927,"De Wette, Wilhelm Martin Leberecht","De Wette, Wilhelm Martin Leberecht",1780-1849.,1841,1841,s,1841,,"Boston;Hilliard, Gray;1841.",1841,...,NaN,22633272,mau,8449523,v.2,2.0,"Theodore : | or, The skeptic's conversion / | ...",NaN,"Theodore : or, The skeptic's conversion",1


#### add copy counts

Before we write out the dataframe, add columns reflecting the number of copies collapsed into each record.

In [107]:
def get_copy_count(idx, dictionary):
    return dictionary[idx]

deduped = deduped.assign(allcopiesofwork = deduped.apply(lambda row: get_copy_count(row.name, authtitlecopies), axis = 1))
deduped = deduped.assign(copiesin25yrs = deduped.apply(lambda row: get_copy_count(row.name, copiesin25yrs), axis = 1))



In [108]:
print(deduped.columns)

Index(['oldauthor', 'author', 'authordate', 'inferreddate', 'latestcomp',
       'datetype', 'startdate', 'enddate', 'imprint', 'imprintdate',
       'contents', 'genres', 'subjects', 'geographics', 'locnum', 'oclc',
       'place', 'recordid', 'enumcron', 'volnum', 'title', 'parttitle',
       'shorttitle', 'instances', 'allcopiesofwork', 'copiesin25yrs'],
      dtype='object')


In [109]:
# sort rows
deduped.sort_values(by = ['inferreddate', 'recordid', 'volnum'], inplace = True)

# put columns in desired order (title last)
deduped = deduped[['oldauthor', 'author', 'authordate', 'inferreddate',
       'latestcomp', 'datetype', 'startdate', 'enddate', 'imprint',
       'imprintdate', 'contents', 'genres', 'subjects', 'geographics',
       'locnum', 'oclc', 'place', 'recordid', 'instances', 'allcopiesofwork',
       'copiesin25yrs', 'enumcron', 'volnum', 'title',
       'parttitle', 'shorttitle']]

# write to file
deduped.to_csv('newworkmeta.tsv', sep = '\t', index = True)